In [1]:
import os

WIKIPEDIA_ROOT = "/media/jeff/Data/data/wikipedia2"
WIKIPEDIA_DL = os.path.join(WIKIPEDIA_ROOT, 'dl')

In [ ]:
import bs4
import os
from urllib.parse import urlparse
import urllib
import posixpath
from bs4 import BeautifulSoup

WIKIPEDIA_URL = 'https://dumps.wikimedia.org/'
WIKIPEDIA_LANG = 'enwiki'

def get_wikidump_available(wiki_url=WIKIPEDIA_URL,wiki_lang=WIKIPEDIA_LANG):
    base_url = posixpath.join(wiki_url, wiki_lang)
    index = urllib.request.urlopen(base_url).read()
    soup_index = BeautifulSoup(index, 'html.parser')
    # Find the links on the page
    return [a['href'] for a in soup_index.find_all('a') if 
             a.has_attr('href')]

def get_latest_wikidump(dumps):
    lst = []
    for dump in dumps:
        try:
            idx = dump.index('/')
            
            if idx != -1:
                dump = dump[:idx]
            
            lst.append(int(dump))
        except ValueError:
            pass
        
    return max(lst)

def wikidump_download(wiki_url=WIKIPEDIA_URL,wiki_lang=WIKIPEDIA_LANG, timestamp=None):
    if not timestamp:
        timestamp = get_latest_wikidump(get_wikidump_available(wiki_url,wiki_lang))
    
    dump_url = posixpath.join(wiki_url, wiki_lang, str(timestamp))
    print(dump_url)
    dump_html = urllib.request.urlopen(dump_url).read()
    soup_dump = BeautifulSoup(dump_html, 'html.parser')
    files = soup_dump.find_all('li', {'class': 'file'})
    
    for items in soup_dump.find_all('li', {'class': 'file'}):
        for child in items.children:
            if isinstance(child,bs4.element.Tag):
                f = child.get('href', default=None)
                if f and '-stub' not in f and '-current' in f and 'xml-' in f:
                    url = urllib.parse.urljoin(WIKIPEDIA_URL, f)
                    a = urlparse(url)
                    target_file = os.path.join(WIKIPEDIA_DL,os.path.basename(a.path))
                    print(url)
                    urllib.request.urlretrieve(url, target_file)
    

wikidump_download()

https://dumps.wikimedia.org/enwiki/20210320
https://dumps.wikimedia.org/enwiki/20210320/enwiki-20210320-pages-meta-current1.xml-p1p41242.bz2
https://dumps.wikimedia.org/enwiki/20210320/enwiki-20210320-pages-meta-current2.xml-p41243p151573.bz2


# Choose the Dump to Download

In [ ]:
dump_url = posixpath.join(base_url, 'latest')

print(dump_url)
# Retrieve the html
dump_html = urllib.request.urlopen(dump_url).read()
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')
# Find list elements with the class file
files = soup_dump.find_all('li', {'class': 'file'})

In [ ]:
files

In [ ]:
for items in soup_dump.find_all('li', {'class': 'file'}):
    for child in items.children:
        if isinstance(child,bs4.element.Tag):
            f = child.get('href', default=None)
            if f and '-stub' not in f and '-current' in f and 'xml-' in f:
                url = urllib.parse.urljoin(WIKIPEDIA_URL, f)
                a = urlparse(url)
                target_file = os.path.join(WIKIPEDIA_DL,os.path.basename(a.path))
                print(url)
                urllib.request.urlretrieve(url, target_file)
